In [1]:
#Executar caso não tenha instalado o módulo pybdedata
!pip install pybdedata

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests as req
from requests.models import PreparedRequest
import pandas as pd
import json
import pybdedata.query as bde
import ipywidgets as widgets

In [3]:
fonteDatas = {
    'IBGE':'https://apisidra.ibge.gov.br/values',
    'TSE':'https://sig.tse.jus.br/ords/dwtse/f?p=2001:101',
    'INEP':{
        'Superior':'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_',
        'Básica':'https://download.inep.gov.br/dados_abertos/sinopses_estatisticas/sinopses_estatisticas_censo_escolar_'
    },
}

In [84]:
def formatUnitMilhar(valor):
        try: 
            teste = float(valor)
            numero = str(valor).split('.')
            if len(numero) > 1:
                decimal = ','+numero[1]
            else:
                decimal = ''    
            valor = numero[0]
            if len(valor) <= 3:
                valor = valor
            elif (len(valor) > 3) and len(valor) < 7:
                valor = valor[:-3]+'.'+valor[-3:]
            elif (len(valor) > 6) and  len(valor) < 10:
                valor = valor[:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            elif (len(valor) > 9) and  len(valor) < 13:
                valor = valor[:-9]+'.'+valor[-9:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            elif (len(valor) > 12) and len(valor) < 17:
                valor = valor[:-12]+'.'+valor[-12:-9]+'.'+valor[-9:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            valor = valor+decimal
        except:
            valor = valor
        return valor 
    
def getSource(pesquisa):
     
    #Dados utilizado no BDE-GO
    fontes = {
              'PEVS':{
                  '289':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'144,145','c193':'all'}}, #Produção da Extração Vegetal e da Silvicultura
                  '291':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'142,143','c194':'all'}}#Produção da Extração Vegetal e da Silvicultura
               },
                  
              'PPM':{
                '74':{'colunas':['D1C','D2C','D3C','D4C','V'],'parametros':{'v':'106','c80':'all'}},
                '94':{'colunas':['D1C','D2C','D3C','V'],'parametros':{'v':'107'}},
                '3939':{'colunas':['D1C','D2C','D3C','D4C','V'],'parametros':{'v':'105','c79':'all'}} 
              },
              'PAM':{
                  '839':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c81':'allxt'}}, 
                  '1002':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c81':'allxt'}},
                  '5457':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c782':'allxt'}}
              }
              #'1378':{'colunas':['D2C','D4C','D3N','V'],'parametros':{'v':'93','c1':'all','c2':'all','c287':'all','c455':'all'}}, #Censo Demográfico
              #'2759':{'colunas':['D1C','D7N','V'],'parametros':{'v':'221','c236':'0','c247':'0','c248':'0','c245':'0','c246':'0'}}, #Pesquisa Estatísticas do Registro Civil
              #'5938':{'colunas':['D1C','D2N','V'],'parametros':{'v':'37,498,513,517,6575,525'}}, #Produto Interno Bruto dos Municípios
              #'6579':{'colunas':['D1C','D2N','V'],'parametros':{'v':'9324'}} #Estimativas de População
    }
    
    tbComplete = pd.DataFrame()
    pesq = fontes[pesquisa]
    
    for cod in pesq.keys():
                
        dados = pesq[cod]['parametros']#fontes[cod]['parametros']
        colunas = pesq[cod]['colunas']#fontes[cod]['colunas']
        
        #Configurando os parametros para acessar a API do IBGE
        param = {'t':cod}
        for value in dados.keys():
            param[value] = dados[value]
        param['n3'] = '52'
        param['n6'] = 'in n3 52'    
        print(param)   
        #Preparando a url para extrair os dados    
        url = PreparedRequest()
        url.prepare_url('https://apisidra.ibge.gov.br/values',param) 
        url = str(url.url).replace('&','/').replace('=','/').replace('?','/').replace('+',' ')
            
        try:
            #Extraindo os dados
            data = req.get(url)
            dados = json.loads(str(data.text))
            dfdata = pd.DataFrame(data=dados)
                        
            #Transformação dos dados
            dfdata = dfdata[colunas]
            dfdata.rename(columns = {
                colunas[-3]:'loc_cod_ibge',
                colunas[-2]:'ano'
            },inplace = True)
            dfdata['codFonte'] = dfdata[colunas[:-3]].apply(lambda x: '-'.join(x.values),axis=1)
            dfdata = dfdata.drop(dfdata.index[0])
            dfdata = dfdata[['loc_cod_ibge','codFonte','ano','V']]
        
            #Atualizando a tabela principal
            tbComplete = pd.concat([tbComplete, dfdata])
        except:
            continue
    
        
    #Retornar a tabela estruturda com os dados
    return tbComplete

def transformData(df):
    #Transformação dos dados
    dfdata = dfdata[colunas]
    dfdata.rename(columns = {colunas[-2]:'loc_nome'},inplace = True)
    dfdata['codFonte'] = dfdata[colunas[:-2]].apply(lambda x: '-'.join(x.values),axis=1)
    dfdata = dfdata.drop(dfdata.index[0])
    dfdata = dfdata[['loc_nome','codFonte','Ano','V']]
    return 

In [85]:
#'Raspando' os dados do IBGE
pesquisa = 'PEVS'
dados = getSource(pesquisa)
dados



{'t': '289', 'v': '144,145', 'c193': 'all', 'n3': '52', 'n6': 'in n3 52'}
{'t': '291', 'v': '142,143', 'c194': 'all', 'n3': '52', 'n6': 'in n3 52'}


,loc_cod_ibge,codFonte,ano,V
1,52,144-0,2024,..
2,5200050,144-0,2024,..
3,5200100,144-0,2024,..
4,5200134,144-0,2024,..
5,5200159,144-0,2024,..
...,...,...,...,...
6596,5221809,143-3463,2024,-
6597,5221908,143-3463,2024,-
6598,5222005,143-3463,2024,-
6599,5222054,143-3463,2024,-


In [86]:
len(dados.codFonte.unique())

152

In [87]:
#Acessando a tabela da relação das variáveis do BDE com o IBGE
src = r'Z:/GEDE/BDE - Banco de Dados/tb_bde-cod-ibge.xlsx'
dfvar = pd.read_excel(src)
dfvar = dfvar[dfvar['codFonte'].notnull()]
dfvar['codFonte'] = dfvar['codFonte'].apply(lambda x: str(x))
dfvar

,var_cod,codFonte,varNome
0,1,93-0-0-0-0,População Censitária - Total
1,2,93-1-0-0-0,População Censitária - Urbana
2,3,93-2-0-0-0,População Censitária - Rural
3,4,93-0-4-0-0,População Censitária - Masculina
4,5,93-0-5-0-0,População Censitária - Feminina
...,...,...,...
433,503,142-3457,Silvicultura - Madeira em Tora - Quantidade
434,504,143-3457,Silvicultura - Madeira em Tora - Valor
435,505,143-0,Silvicultura - Total - Valor
699,829,144-39409,Extração Vegetal - Pequi - Quantidade


In [88]:
dfvar.codFonte.unique()

array(['93-0-0-0-0', '93-1-0-0-0', '93-2-0-0-0', '93-0-4-0-0',
       '93-0-5-0-0', '93-1-4-0-0', '93-1-5-0-0', '93-2-4-0-0',
       '93-2-5-0-0', '2047-0', '2047-4', '2047-5', '2047-1', '2047-2',
       '9324', '105-2670', '105-32794', '107', '216-40129', '214-40129',
       '216-40092', '214-40092', '216-40099', '214-40099', '216-40100',
       '214-40100', '216-40101', '214-40101', '216-40102', '214-40102',
       '216-40136', '214-40136', '216-40104', '214-40104', '216-40105',
       '214-40105', '216-40137', '214-40137', '216-40139', '214-40139',
       '216-40106', '214-40106', '216-40107', '214-40107', '216-40109',
       '214-40109', '216-40145', '214-40145', '216-40110', '214-40110',
       '216-40112', '214-40112', '216-117991', '214-117991', '216-117992',
       '214-117992', '216-117993', '214-117993', '216-40148', '214-40148',
       '216-40113', '214-40113', '216-40114', '214-40114', '216-40149',
       '214-40149', '216-40151', '214-40151', '216-40152', '214-40152',
    

In [89]:
bdeObj = bde.BDEquery()
locations = pd.DataFrame(data=bdeObj.getLocations)
newrows = pd.DataFrame(data={'loc_cod':[1000],'loc_nome':['ESTADO DE GOIÁS'],'loc_cod_ibge':['52']})
locations = pd.concat([locations,newrows])
locations.reset_index(inplace=True)
locations


,index,loc_cod,loc_nome,loc_cod_ibge
0,0,8,Abadia de Goiás,5200050
1,1,9,Abadiânia,5200100
2,2,10,Acreúna,5200134
3,3,11,Adelândia,5200159
4,4,12,Água Fria de Goiás,5200175
...,...,...,...,...
242,242,243,Vianópolis,5222005
243,243,244,Vicentinópolis,5222054
244,244,245,Vila Boa,5222203
245,245,246,Vila Propício,5222302


In [90]:
dados = dados.merge(locations,on='loc_cod_ibge')
dados

,loc_cod_ibge,codFonte,ano,V,index,loc_cod,loc_nome
0,52,144-0,2024,..,0,1000,ESTADO DE GOIÁS
1,5200050,144-0,2024,..,0,8,Abadia de Goiás
2,5200100,144-0,2024,..,1,9,Abadiânia
3,5200134,144-0,2024,..,2,10,Acreúna
4,5200159,144-0,2024,..,3,11,Adelândia
...,...,...,...,...,...,...,...
32947,5221809,143-3463,2024,-,239,240,Urutaí
32948,5221908,143-3463,2024,-,241,242,Varjão
32949,5222005,143-3463,2024,-,242,243,Vianópolis
32950,5222054,143-3463,2024,-,243,244,Vicentinópolis


In [91]:
dados.loc_cod_ibge.unique()

array(['52', '5200050', '5200100', '5200134', '5200159', '5200175',
       '5200209', '5200308', '5200506', '5200555', '5200605', '5200803',
       '5200829', '5200852', '5200902', '5201108', '5201207', '5201306',
       '5201405', '5201454', '5201504', '5201603', '5201702', '5201801',
       '5202155', '5202353', '5202502', '5202601', '5202809', '5203104',
       '5203203', '5203302', '5203401', '5203500', '5203559', '5203575',
       '5203609', '5203807', '5203906', '5203939', '5203962', '5204003',
       '5204102', '5204201', '5204250', '5204300', '5204409', '5204508',
       '5204557', '5204607', '5204656', '5204706', '5204805', '5204854',
       '5204904', '5204953', '5205000', '5205059', '5205109', '5205208',
       '5205307', '5205406', '5205455', '5205471', '5205497', '5205513',
       '5205521', '5205703', '5205802', '5205901', '5206206', '5206305',
       '5206404', '5206503', '5206602', '5206701', '5206800', '5206909',
       '5207105', '5207253', '5207352', '5207402', '5207

In [92]:
#Transformação dos dados (Unir e Criação de novos atributos)
datamerge = dados.merge(dfvar,on='codFonte')
datamerge['Valor'] = datamerge['V'].apply(lambda x: formatUnitMilhar(x))
#datamerge = datamerge.merge(dfvar,on='codFonte')
datamerge

,loc_cod_ibge,codFonte,ano,V,index,loc_cod,loc_nome,var_cod,varNome,Valor
0,52,144-39409,2024,2897,0,1000,ESTADO DE GOIÁS,829,Extração Vegetal - Pequi - Quantidade,2.897
1,5200050,144-39409,2024,0,0,8,Abadia de Goiás,829,Extração Vegetal - Pequi - Quantidade,0
2,5200100,144-39409,2024,...,1,9,Abadiânia,829,Extração Vegetal - Pequi - Quantidade,...
3,5200134,144-39409,2024,-,2,10,Acreúna,829,Extração Vegetal - Pequi - Quantidade,-
4,5200159,144-39409,2024,...,3,11,Adelândia,829,Extração Vegetal - Pequi - Quantidade,...
...,...,...,...,...,...,...,...,...,...,...
3241,5221809,143-3457,2024,1115,239,240,Urutaí,504,Silvicultura - Madeira em Tora - Valor,1.115
3242,5221908,143-3457,2024,-,241,242,Varjão,504,Silvicultura - Madeira em Tora - Valor,-
3243,5222005,143-3457,2024,1251,242,243,Vianópolis,504,Silvicultura - Madeira em Tora - Valor,1.251
3244,5222054,143-3457,2024,-,243,244,Vicentinópolis,504,Silvicultura - Madeira em Tora - Valor,-


In [93]:
#Quantidade de variáveis
len(datamerge.var_cod.unique())

16

In [94]:
db = datamerge.copy()
fieldValue = 'd_'+str(db.ano.unique()[0])
db.rename(columns={'Valor':fieldValue},inplace=True)
db = db[['loc_nome','loc_cod','var_cod',fieldValue]]
db['loc_nome'] = db['loc_nome'].apply(lambda x: x.upper())
db


,loc_nome,loc_cod,var_cod,d_2024
0,ESTADO DE GOIÁS,1000,829,2.897
1,ABADIA DE GOIÁS,8,829,0
2,ABADIÂNIA,9,829,...
3,ACREÚNA,10,829,-
4,ADELÂNDIA,11,829,...
...,...,...,...,...
3241,URUTAÍ,240,504,1.115
3242,VARJÃO,242,504,-
3243,VIANÓPOLIS,243,504,1.251
3244,VICENTINÓPOLIS,244,504,-


In [95]:
db.var_cod.unique()

array([829, 492, 494, 496, 498, 830, 493, 495, 497, 499, 501, 503, 505,
       500, 502, 504])

In [96]:
#Verificar valores nulos na base de dados
datamerge.isnull().sum()

loc_cod_ibge    0
codFonte        0
ano             0
V               0
index           0
loc_cod         0
loc_nome        0
var_cod         0
varNome         0
Valor           0
dtype: int64

In [97]:
#Verificar se todos os municipios foram preenchidos pela quantidade de variaveis
report = {}
for j in datamerge.loc_nome.unique():
    data = datamerge[datamerge.loc_nome==j]
    qtd = len(data)
    report[j] = qtd

report = sorted(report.items(), key=lambda x:x[1],reverse=True)
report

[('ESTADO DE GOIÁS', 16),
 ('Abadia de Goiás', 16),
 ('Abadiânia', 16),
 ('Acreúna', 16),
 ('Adelândia', 16),
 ('Água Fria de Goiás', 16),
 ('Água Limpa', 16),
 ('Alexânia', 16),
 ('Alto Paraíso de Goiás', 16),
 ('Anápolis', 16),
 ('Anhanguera', 16),
 ('Anicuns', 16),
 ('Aparecida de Goiânia', 16),
 ('Aparecida do Rio Doce', 16),
 ('Aporé', 16),
 ('Aragoiânia', 16),
 ('Araguapaz', 16),
 ('Avelinópolis', 16),
 ('Barro Alto', 16),
 ('Bela Vista de Goiás', 16),
 ('Bom Jesus de Goiás', 16),
 ('Bonfinópolis', 16),
 ('Bonópolis', 16),
 ('Buriti Alegre', 16),
 ('Cabeceiras', 16),
 ('Cachoeira Alta', 16),
 ('Cachoeira de Goiás', 16),
 ('Cachoeira Dourada', 16),
 ('Caçu', 16),
 ('Caiapônia', 16),
 ('Caldas Novas', 16),
 ('Caldazinha', 16),
 ('Campestre de Goiás', 16),
 ('Campinaçu', 16),
 ('Campinorte', 16),
 ('Campo Alegre de Goiás', 16),
 ('Campo Limpo de Goiás', 16),
 ('Campos Verdes', 16),
 ('Catalão', 16),
 ('Caturaí', 16),
 ('Cezarina', 16),
 ('Chapadão do Céu', 16),
 ('Cidade Ocidental',

In [100]:
#exporta para csv
db.to_csv('Z:/GEDE/BDE - Banco de Dados/2025/PAM_PPM_PEVS/PVES.csv', encoding='utf-8-sig', index=False)